In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [6]:
import csv
from collections import defaultdict

def create_text_files_from_csv(csv_file_path):
    # Read the CSV file and group rows by transcript_id
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)  # Skip the header row

        conversations = defaultdict(list)
        for row in csv_reader:
            transcript_id = row[0]
            interlocutor = row[6]  # Column for interlocutor
            utterance_text = row[8]  # Column for utterance text
            formatted_utterance = f"{interlocutor.capitalize()}: {utterance_text}\n"
            conversations[transcript_id].append(formatted_utterance)

    # Create a text file for each conversation
    for transcript_id, utterances in conversations.items():
        with open(f'./data/conversations/conversation_{transcript_id}.txt', 'w', encoding='utf-8') as file:
            for utterance in utterances:
                file.write(utterance)

# Replace 'path_to_your_csv.csv' with the path to your CSV file
create_text_files_from_csv('./data/therapy_data/dataset.csv')


In [22]:
import csv
import json
import ast

def create_text_files_from_csv(csv_file_path, output_directory):
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for row in csv_reader:
            conversation_id = row['id']
            # Use ast.literal_eval to safely evaluate the string as a Python literal
            conversation_data = ast.literal_eval(row['conversations'])

            with open(f'{output_directory}/conversation_{conversation_id}.txt', 'w', encoding='utf-8') as file:
                for message in conversation_data:
                    speaker = "Therapist" if message['from'] == 'gpt' else "Patient"
                    file.write(f"{speaker}: {message['value']}\n")

# Replace with your actual CSV file path and output directory
csv_file_path = './data/therapy_data/train.csv'
output_directory = './data/conversations_synthetic'
create_text_files_from_csv(csv_file_path, output_directory)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)

In [27]:
import csv

def parse_conversation_string(conversation_str):
    # Remove the outer brackets and split the string into individual message strings
    messages_str = conversation_str.strip("[]").split("}, {")
    messages = []

    for message_str in messages_str:
        # Correctly format each message string
        message_str = message_str.replace("{'from': '", "").replace("', 'value': '", ": ", 1).replace("'}", "")
        if message_str.startswith("{"):
            message_str = message_str[1:]
        if message_str.endswith("}"):
            message_str = message_str[:-1]

        # Split the formatted string into speaker and message
        speaker, message = message_str.split(": ", 1)
        speaker = "Therapist" if speaker == 'gpt' else "Patient"
        messages.append(f"{speaker}: {message}")

    return messages

def create_text_files_from_csv(csv_file_path, output_directory):
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for row in csv_reader:
            conversation_id = row['id']
            conversation_data = parse_conversation_string(row['conversations'])

            with open(f'{output_directory}/conversation_{conversation_id}.txt', 'w', encoding='utf-8') as file:
                for message in conversation_data:
                    file.write(message + "\n")


                    # Replace with your actual CSV file path and output directory
csv_file_path = './data/therapy_data/train.csv'
output_directory = './data/conversations_synthetic'
create_text_files_from_csv(csv_file_path, output_directory)